In [1]:
import tensorflow as tf
import numpy as np
import tensorflow_probability as tfp
from tensorflow import keras
import pickle as pkl

In [5]:
file = open('8x8lattices.pkl','rb')
lattice_set = pkl.load(file)

t = 0.05 + 5*(2.0/32)
print(t)
#10k samples each of shape (8,8)
lattices = np.array(lattice_set[5])
print(lattices.shape)

UnpicklingError: ignored

In [ ]:
def checkerboard(height, width, reverse=False, dtype=tf.float32):
  checkerboard = [[((i % 2) + j) % 2 for j in range(width)] for i in range(height)]
  checkerboard = tf.convert_to_tensor(checkerboard, dtype = dtype)
  if reverse:
      checkerboard = 1 - checkerboard
  checkerboard = tf.reshape(checkerboard, (1,height,width,1))
  return tf.cast(checkerboard, dtype=dtype)

In [ ]:
def periodic_padding(x, padding=1):
  '''
  x: shape (batch_size, d1, d2)
  return x padded with periodic boundaries. i.e. torus or donut
  '''
  d1 = x.shape[1] # dimension 1: height
  d2 = x.shape[2] # dimension 2: width
  p = padding
  # assemble padded x from slices
  #            tl,tc,tr
  # padded_x = ml,mc,mr
  #            bl,bc,br
  top_left = x[:, -p:, -p:] # top left
  top_center = x[:, -p:, :] # top center
  top_right = x[:, -p:, :p] # top right
  middle_left = x[:, :, -p:] # middle left
  middle_center = x # middle center
  middle_right = x[:, :, :p] # middle right
  bottom_left = x[:, :p, -p:] # bottom left
  bottom_center = x[:, :p, :] # bottom center
  bottom_right = x[:, :p, :p] # bottom right
  top = tf.concat([top_left, top_center, top_right], axis=2)
  middle = tf.concat([middle_left, middle_center, middle_right], axis=2)
  bottom = tf.concat([bottom_left, bottom_center, bottom_right], axis=2)
  padded_x = tf.concat([top, middle, bottom], axis=1)
  return padded_x

In [ ]:
filters = 64

def Coupling(input_shape):
  input = keras.layers.Input(shape=input_shape)

  layer1 = keras.layers.Conv2D(filters,3, activation="relu",padding = 'valid',name = 'layer1')(periodic_padding(input))
  layer2 = keras.layers.Conv2D(filters,3, activation="relu",padding = 'valid',name = 'layer2')(periodic_padding(layer1))
  t_layer= keras.layers.Conv2D(1,3,padding = 'valid',name = 't_layer')(periodic_padding(layer2))
  s_layer= keras.layers.Conv2D(1,3,activation = 'tanh',padding = 'valid', name = 's_layer')(periodic_padding(layer2))

  return keras.Model(inputs=input, outputs=[s_layer, t_layer])

In [ ]:
class SimpleNormal:
  def __init__(self, loc, var):
    self.dist = tfp.distributions.Normal(tf.reshape(loc,(-1,)), tf.reshape(var,(-1,)))
    self.shape = loc.shape
  def log_prob(self, x):
    logp = self.dist.log_prob(tf.reshape(x,(x.shape[0], -1)))
    return tf.reduce_sum(logp, axis=1)
  def sample_n(self, batch_size , seed = None):
    x = self.dist.sample((batch_size,),seed = seed)
    return tf.reshape(x,(batch_size, *self.shape))

In [ ]:
class RealNVP(keras.Model):
  def __init__(self, num_coupling_layers,input_shape , data_constraint):
      super().__init__()
      self.num_coupling_layers = num_coupling_layers
      self.distribution = SimpleNormal(tf.zeros((8,8)), tf.ones((8,8)))
      self.masks = [checkerboard(input_shape[0],input_shape[1], reverse=False),checkerboard(input_shape[0],input_shape[1], reverse=True)]*(num_coupling_layers // 2)
      self.loss_tracker = keras.metrics.Mean(name="loss")
      self.layers_list = [Coupling(input_shape) for i in range(num_coupling_layers)]
      self.data_constraint = data_constraint

  def call(self, x, forward=True):
    if forward:
      alpha = tf.constant(self.data_constraint)
      logq = self.distribution.log_prob(tf.reshape(x,(-1,8,8)))
      x,ldj1 = self.forward(x)
      ldj2 = tf.math.softplus(x) + tf.math.softplus(-x) + tf.math.log(tf.constant(1.-2*self.data_constraint))
      ldj2 = tf.reduce_sum(ldj2,[1,2,3])
      logq = logq - ldj1 + ldj2
      x   = (tf.math.sigmoid(x) - alpha)/(1-2*alpha)
      return x, logq
    else:
      x = self.data_constraint + (1-2*self.data_constraint)*x
      x = tf.math.log(x/(1.-x))
      # Save log-determinant of Jacobian of initial transform
      ldj1 = tf.math.softplus(x) + tf.math.softplus(-x) + tf.math.log(tf.constant(1.-2*self.data_constraint))
      ldj1 = tf.reduce_sum(ldj1,[1,2,3])
      x,ldj2 = self.reverse(x)
      logq = self.distribution.log_prob(tf.reshape(x,(-1,8,8)))
      logq = logq + ldj2 + ldj1
      return x , logq

  def forward(self, x):
    ldj = 0
    for i in range(self.num_coupling_layers):
      x_frozen = x * self.masks[i]
      reversed_mask = 1 - self.masks[i]
      x_active = x * reversed_mask
      s, t = self.layers_list[i](x_frozen)
      s *= reversed_mask
      t *= reversed_mask

      fx = t + x_active *tf.exp(s) + x_frozen
      ldj += tf.reduce_sum(s, [1,2,3])
      x = fx
    return x, ldj

  def reverse(self, fx):
    ldj = 0
    for i in reversed(range(self.num_coupling_layers)):
      fx_frozen = fx*self.masks[i]
      reversed_mask = 1 - self.masks[i]
      fx_active = fx*reversed_mask
      s, t = self.layers_list[i](fx_frozen)
      s *= reversed_mask
      t *= reversed_mask

      x = (fx_active - t) *tf.exp(-s) + fx_frozen
      ldj -= tf.reduce_sum(s, [1,2,3])
      fx = x
    return fx,ldj